# This is the Jupyter Notebook for the Iterative Data Cleaning Process

In [3]:
!unzip /content/Cropped_Data_4000_Three.zip -d /content/

Archive:  /content/Cropped_Data_4000_Three.zip
   creating: /content/Cropped_Data_4000_Three/Algae/
  inflating: /content/Cropped_Data_4000_Three/Algae/420041461011010173.jpg  
  inflating: /content/Cropped_Data_4000_Three/Algae/420041461011091120.jpg  
  inflating: /content/Cropped_Data_4000_Three/Algae/420041461011243323.jpg  
  inflating: /content/Cropped_Data_4000_Three/Algae/420041461011253896.jpg  
  inflating: /content/Cropped_Data_4000_Three/Algae/420041461011399166.jpg  
  inflating: /content/Cropped_Data_4000_Three/Algae/4200414610114801502.jpg  
  inflating: /content/Cropped_Data_4000_Three/Algae/4200414610116111778.jpg  
  inflating: /content/Cropped_Data_4000_Three/Algae/4200414610116611504.jpg  
  inflating: /content/Cropped_Data_4000_Three/Algae/42004146101311474.jpg  
  inflating: /content/Cropped_Data_4000_Three/Algae/42004146101399487.jpg  
  inflating: /content/Cropped_Data_4000_Three/Algae/420041461017301695.jpg  
  inflating: /content/Cropped_Data_4000_Three/Algae/

In [1]:
#Import Statements
import numpy as np
import math
import matplotlib.pyplot as plt
import torchvision
import torch
from torchvision import datasets, transforms
from torch.utils.data import random_split
import torch.nn as nn
import torchvision.models as models
from tqdm import tqdm
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
!pip install timm
import timm
import os
import csv
import shutil
import random
from tqdm import tqdm

Defaulting to user installation because normal site-packages is not writeable


In [2]:
#This is one big cell of all the helper functions we need

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform
    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return x, y
    def __len__(self):
        return len(self.subset)
def prepare_and_get_loaders(validation_split, batch_size, num_workers, dir_path):
    """
    This function loads and partitions our image data.

    val_split: proportion of the data that will be in the validation split
    batch_size: size of batches
    num_workers: number of parallel processors
    dir_path: directory path of the data
    """

    #Transforms to apply to training and test data
    data_transforms = {
        'train': transforms.Compose([
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ]),

        'test': transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])
    }
    full_dataset = datasets.ImageFolder(dir_path)

    m = len(full_dataset)
    val_size = int(validation_split * m)
    train_size = m - val_size

    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
    train_dataset = MyDataset(train_dataset, transform=data_transforms['train'])
    val_dataset = MyDataset(val_dataset, transform=data_transforms['test'])

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

    return train_loader, val_loader

def finetune(model, train_loader, val_loader, learning_rate, num_epochs, L2, momentum1, momentum2, stepsize, gamma):
    """

    Finetunes the given model on the loader data, and returns the trained model as well as the losses
    and final validation accuracy for plotting.

    model: the Pytorch model
    train_loader: the loader for the training data
    val_loader: the loader for the validation data
    learning_rate: alpha, the learning rate
    num_epochs: the number of epochs to train for
    L2: the L2 regularization value
    momentum1: the first momentum value
    momentum2: the second momentum value
    stepsize: the number of epochs between lowering the learning rate
    gamma: the ratio to multiply the learning rate at each step size

    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"My Device is {device}")
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()

    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=L2, betas = (momentum1, momentum2))

    scheduler = StepLR(optimizer, step_size=stepsize, gamma=gamma)

    hist = {'train': [], 'val': []} # History of training and validation losses

    for epoch in tqdm(range(num_epochs), desc = "Training Epoch: "):
        model.train() #Training mode
        total_loss = 0.0
        for inputs, labels in train_loader:
            #put stuff on device
            inputs = inputs.to(device)
            labels = labels.to(device)

            #zero optimizer
            optimizer.zero_grad()

            #get outputs + train
            outputs = model(inputs)

            #get loss and do backward
            loss = criterion(outputs, labels)
            loss.backward()

            #step
            optimizer.step()

            total_loss += loss.item()

            # Free memory
            if device == "cuda":
              del inputs, labels, outputs, loss

        avg_train_loss = total_loss / len(train_loader)
        hist['train'].append(avg_train_loss)

        #Validation Mode Now
        model.eval()
        total_loss = 0.0

        #keep track of correct labels so we can get the val accuracy
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                #put on device
                inputs = inputs.to(device)
                labels = labels.to(device)

                #get outputs
                outputs = model(inputs)

                #get loss
                loss = criterion(outputs, labels)
                total_loss += loss.item()

                #https://www.digitalocean.com/community/tutorials/pytorch-torch-max
                max_element, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                # Free memory
                if device == "cuda":
                  del inputs, labels, outputs, loss

        avg_val_loss = total_loss / len(val_loader)
        hist['val'].append(avg_val_loss)
        val_accuracy = 100 * correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')
        scheduler.step()

        if device == "cuda":
          torch.cuda.empty_cache() #cuda memory was filling up
    #final val accuracy
    final_val_accuracy = 100*correct/total

    return model, hist, final_val_accuracy

def load_model(arch, dropout, num_classes):
    """

    Loads a pretrained model, but with the final output layer removed and replaced with a randomly
    initialized output layer that will classify among the coral. 

    """

    #select the model we want
    if arch == "resnet18":
        model = models.resnet18(weights="ResNet18_Weights.DEFAULT")
        num_ftrs = model.fc.in_features
        model.fc = nn.Sequential(nn.Dropout(dropout), nn.Linear(num_ftrs, num_classes))

    elif arch == "resnet34":
        model = models.resnet34(weights="DEFAULT")
        num_ftrs = model.fc.in_features
        model.fc = nn.Sequential(nn.Dropout(dropout), nn.Linear(num_ftrs, num_classes))

    elif arch == "mobile":
        model = models.mobilenet_v2(weights="MobileNet_V2_Weights.DEFAULT")
        num_ftrs = model.classifier[1].in_features
        model.classifier = nn.Sequential(nn.Dropout(dropout), nn.Linear(num_ftrs, num_classes))

    elif arch == "alexnet":
            model = models.alexnet(weights="AlexNet_Weights.DEFAULT")
            num_ftrs = model.classifier[1].in_features
            model.classifier[6] = nn.Sequential(nn.Dropout(dropout), nn.Linear(num_ftrs, num_classes))

    elif arch == "vgg16":
        model = models.vgg16(weights="VGG16_Weights.DEFAULT")
        num_ftrs = model.classifier[0].in_features
        model.classifier[6] = nn.Sequential(nn.Dropout(dropout), nn.Linear(num_ftrs, num_classes))

    elif arch == "densenet":
        model = models.densenet121(weights="DenseNet121_Weights.DEFAULT")
        num_ftrs = model.classifier.in_features
        model.classifier = nn.Sequential(nn.Dropout(dropout), nn.Linear(num_ftrs, num_classes))

    elif arch == "inception":
        model = models.inception_v3(weights="Inception_V3_Weights.DEFAULT")
        num_ftrs = model.fc.in_features
        model.fc = nn.Sequential(nn.Dropout(dropout), nn.Linear(num_ftrs, num_classes))

    elif arch == "efficientnet":
        model = models.efficientnet_b0(weights="EfficientNet_B0_Weights.DEFAULT")
        num_ftrs = model.classifier[1].in_features
        model.classifier = nn.Sequential(nn.Dropout(dropout), nn.Linear(num_ftrs, num_classes))

    elif arch == "googlenet":
        model = models.googlenet(weights="GoogLeNet_Weights.DEFAULT")
        num_ftrs = model.fc.in_features
        model.fc = nn.Sequential(nn.Dropout(dropout), nn.Linear(num_ftrs, num_classes))

    elif arch == "squeezenet":
        model = models.squeezenet1_0(weights="SqueezeNet1_0_Weights.DEFAULT")
        model.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model.num_classes = num_classes

    elif arch == "vit":
        model = timm.create_model('vit_base_patch16_224', pretrained=True)
        num_ftrs = model.head.in_features
        model.head = nn.Sequential(nn.Dropout(dropout), nn.Linear(num_ftrs, num_classes))
    elif arch == "deit":
        model = timm.create_model('deit_base_patch16_224', pretrained=True)
        num_ftrs = model.head.in_features
        model.head = nn.Sequential(nn.Dropout(dropout), nn.Linear(num_ftrs, num_classes))
        
    elif arch == "mixer":
        model = timm.create_model('mixer_b16_224', pretrained=True)
        num_ftrs = model.head.in_features
        model.head = nn.Sequential(nn.Dropout(dropout), nn.Linear(num_ftrs, num_classes))

    elif arch == "tnt":
        model = timm.create_model('tnt_s_patch16_224', pretrained=True)
        num_ftrs = model.head.in_features
        model.head = nn.Sequential(nn.Dropout(dropout), nn.Linear(num_ftrs, num_classes))
    else:
        print("Incorrect model Architecture selected")
        raise

    return model


def plot_history(hist):
    epochs = range(1, len(hist['train']) + 1)

    # Plot training and validation loss
    plt.figure(figsize=(12, 6))
    plt.plot(epochs, hist['train'], 'b', label='Training loss')
    plt.plot(epochs, hist['val'], 'r', label='Validation loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

def create_labels_csv(data_dir, csv_filename):
    """
    Create a CSV file of the label, name, and also the path of the images in a particular directory
    without editing the images. 

    data_dir: Directory containing the images organized by label.
    csv_filename: Path to the CSV file to be created. Can also overwrite
    """

    with open(csv_filename, mode='w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['image_path', 'image_name', 'label'])

        for label in os.listdir(data_dir):
            label_dir = os.path.join(data_dir, label)
            if os.path.isdir(label_dir):
                for image_file in os.listdir(label_dir):
                    image_path = os.path.join(label_dir, image_file)
                    image_name = os.path.basename(image_path)
                    csv_writer.writerow([image_path, image_name, label])

    print(f"CSV file created at {csv_filename}")

def create_dirty_data(original_dir, dirty_dir, x):
    """
    Create a dataset with x% of the data having randomized labels,
    while the rest retain their original labels.

    original_dir: The directory of the original dataset.
    dirty_dir: The directory to store the dirty dataset.
    x: The proportion of the dataset to be made 'dirty' (0 < x < 1).
    """

    print("Deleting Dirty Data Directory")
    if os.path.exists(dirty_dir):
        shutil.rmtree(dirty_dir)
    print("Writing Dirty Data Directory")
    # Create the dirty directory
    os.makedirs(dirty_dir)

    # Get all image paths and their labels
    images = []
    for class_name in os.listdir(original_dir):
        class_dir = os.path.join(original_dir, class_name)
        if os.path.isdir(class_dir):
            images.extend([(os.path.join(class_dir, img), class_name) for img in os.listdir(class_dir)])

    # Randomly select a subset of images to be 'dirty'
    num_dirty = int(len(images) * x)
    dirty_data = set(random.sample(images, num_dirty))

    # Copy images to the new directory
    class_names = os.listdir(original_dir)
    
    for img_path, label in tqdm(images, desc="Writing Data"):
        if (img_path, label) in dirty_data:
            # Randomize label for dirty data
            new_label = random.choice(class_names)
        else:
            new_label = label

        new_path = os.path.join(dirty_dir, new_label, os.path.basename(img_path))
        os.makedirs(os.path.dirname(new_path), exist_ok=True)
        shutil.copy(img_path, new_path)

        
def calculate_label_accuracy(clean_csv, dirty_csv):
    """
    Calculate the label accuracy of the dirty dataset compared to the clean dataset.

    clean_csv: Path to the CSV file with the original (clean) labels.
    dirty_csv: Path to the CSV file with the dirty labels.
    """

    # Read the CSV files and store labels in dictionaries
    clean_labels = {}
    dirty_labels = {}

    with open(clean_csv, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            clean_labels[row['image_name']] = row['label']

    with open(dirty_csv, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            dirty_labels[row['image_name']] = row['label']

    # Compare labels and calculate accuracy
    correct = 0
    total = len(dirty_labels)

    for image_name, label in dirty_labels.items():
        if image_name not in clean_labels:
            print("Image not found!")
        if clean_labels[image_name] == label:
            correct += 1

    accuracy = correct / total if total > 0 else 0
    return accuracy



class CleaningDataset(torch.utils.data.Dataset): #new dataset that also returns the path to the image
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform
    def __getitem__(self, index):
        x, y = self.subset[index]
        img_path = self.subset.imgs[index][0]
        if self.transform:
            x = self.transform(x)
        return x, y, img_path
    def __len__(self):
        return len(self.subset)



def cleaning_loader(dir_path):
    """
    This function makes our cleaning data loader, and we also need the class names for manual relabelling

    dir_path: directory path of the data
    """

    #Loading transform
    data_transforms = {
        'load': transforms.Compose([
            transforms.Resize((244,244)),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])
    }
    full_dataset = datasets.ImageFolder(dir_path)
    class_names = full_dataset.classes

    load_dataset = CleaningDataset(full_dataset, transform=data_transforms['load'])
    clean_loader = torch.utils.data.DataLoader(load_dataset, shuffle=False, num_workers=0)

    return clean_loader, class_names




def get_misclassified_images(loader, model, class_names, confidence_threshold):
    model.eval()
    misclassified = []

    with torch.no_grad():
        for images, true_labels, paths in tqdm(iter(loader), desc="Evaluating"):
            outputs = model(images)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
            max_probs, predicted_labels = torch.max(probabilities, dim=1)

            for i in range(len(images)):
                if predicted_labels[i] != true_labels[i] and max_probs[i] > confidence_threshold:
                    misclassified.append({
                        'image': images[i],
                        'true_label': true_labels[i],
                        'predicted_label': predicted_labels[i],
                        'confidence': max_probs[i].item(),
                        'path': paths[i]
                    })

    # Sort in descending order cuz we wanna show the worst classified first
    misclassified.sort(key=lambda x: x['confidence'], reverse=True)
    return misclassified



def rewrite_data_with_new_labels(dir_1, dir_2, misclassified_images):
    """
    Copy data from dir_1 to dir_2, changing the labels of misclassified images.

    dir_1: Current directory of the data.
    dir_2: Target directory where data will be rewritten.
    misclassified_images: List of misclassified images and their new labels. Gotten from the function above.
    """

    # Create dir_2 if it doesn't exist
    if not os.path.exists(dir_2):
        os.makedirs(dir_2)

    # Create a set of misclassified image paths for quick lookup
    misclassified_paths = set([item['path'] for item in misclassified_images])

    # Mapping of predicted labels for misclassified images
    predicted_labels = {item['path']: item['predicted_label'] for item in misclassified_images}

    # Iterate over dir_1 and copy files to dir_2
    for root, dirs, files in os.walk(dir_1):
        for file in tqdm(files, desc="Copying data"):
            src_path = os.path.join(root, file)

            # Determine the target directory based on whether the image is misclassified
            if src_path in misclassified_paths:
                # Use the new predicted label directory for misclassified images
                new_label_dir = os.path.join(dir_2, predicted_labels[src_path])
            else:
                # Use the same directory structure as dir_1 for correctly classified images
                rel_path = os.path.relpath(root, dir_1)
                new_label_dir = os.path.join(dir_2, rel_path)

            # Create target directory if it doesn't exist
            os.makedirs(new_label_dir, exist_ok=True)

            # Copy file to the new directory
            shutil.copy(src_path, os.path.join(new_label_dir, file))





# Below, we perform the automatic data cleaning process

In [3]:
#Define the clean, first dirty, and second dirty directories. 
clean_dir = r"C:\Users\Brandon\Desktop\Classes\E208\Homework\E208_Final\Automatic Data Cleaning\Cropped_Data_400"
dirty_dir_1 = r"C:\Users\Brandon\Desktop\Classes\E208\Homework\E208_Final\Automatic Data Cleaning\Dirty Data 1"
dirty_dir_2 =  r"C:\Users\Brandon\Desktop\Classes\E208\Homework\E208_Final\Automatic Data Cleaning\Dirty Data 2"

#Define the CSVs for each one
clean_csv = "Clean_Labels.csv"
dirty_csv_1 = "Dirty_Labels_1.csv"
dirty_csv_2 = "Dirty_Labels_2.csv"

# Define the starting dirty proportion of the dataset we will be using
dirty_proportion = 0.1 

# Initialize our datasets and csvs before going into the loop
create_labels_csv(clean_dir, clean_csv)
create_dirty_data(clean_dir, dirty_dir_1, dirty_proportion)
create_labels_csv(dirty_dir_1, dirty_csv_1)

# Double check our dirty accuracy - should be close to: 1 - 2/3*dirty_proportion
print(f"Our dirty proportion should be approximately: {1-2/3*dirty_proportion}")
label_accuracy = calculate_label_accuracy(clean_csv, dirty_csv_1)
print(f"Label accuracy of the dirty dataset: {label_accuracy}")

#Initialize what our current dirty dataset is for the iterative loop, and other starting variables
current_dirty_dataset = 1
confidence_threshold = 0.5 #If we are over 50% confident that the image is the other label, change the label
num_iterations = 3
label_accuracy_history = []

#Training params
validation_split = 0.2
batch_size = 64
num_workers = 0

arch = "resnet18" #(Model architecture we'll use)
dropout = 0
num_classes = 3

learning_rate = 0.0001
num_epochs = 4 
L2 = 10**(-6)
momentum1 = 0.8
momentum2 = 0.9
stepsize = 3
gamma = 0.1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


CSV file created at Clean_Labels.csv
Deleting Dirty Data Directory
Writing Dirty Data Directory


Writing Data: 100%|█████████████████████████████████████████████████████████████████| 385/385 [00:00<00:00, 842.91it/s]

CSV file created at Dirty_Labels_1.csv
Our dirty proportion should be approximately: 0.9333333333333333
Label accuracy of the dirty dataset: 0.9272727272727272


## We perform the data cleaning process below. 
### It occurs in the following process:

1. Train a model on the current dirty dataset

2. Evaluate that model on the current dirty dataset. If the model is over the confidence threshold on an image, change the label and write the new dataset to the other dirty dataset. Log the new dataset's label accuracy. The other dirty dataset is the new current dataset. 

3. Repreat steps 1-2 for the desired number of iterations. 

In [5]:
for it in range(num_iterations):
    #Step 1: Train a model on the current dirty dataset
    if current_dirty_dataset == 1:
        dir_path = dirty_dir_1
        new_dir_path = dirty_dir_2
    elif current_dirty_dataset == 2:
        dir_path = dirty_dir_2
        new_dir_path = dirty_dir_1
    else:
        raise
        
    #Track the progress of the label accuracy
    label_accuracy = calculate_label_accuracy(clean_csv, dir_path)
    label_accuracy_history.append(label_accuracy)
        
    #Make our dataset w/ the dirty data
    train_loader, val_loader = prepare_and_get_loaders(validation_split, batch_size, num_workers, dir_path)
    
    #Make our model with the best one we found
    model = load_model(arch, dropout, num_classes)
    
    #Train the model
    model, hist, final_val_accuracy = finetune(model,
                                          train_loader,
                                          val_loader,
                                          learning_rate,
                                          num_epochs,
                                          L2,
                                          momentum1,
                                          momentum2,
                                          stepsize,
                                          gamma)
    
    
    #Step 2: Evaluate the model on the dataset and keep track of the ones above the confidence threshold.
    #Then, write the data to the new directory with the only change being that the images the model wants to relabel are 
    #relabelled
    clean_loader, class_names = cleaning_loader(dir_path)
    misclassified_images = get_misclassified_images(clean_loader, model, class_names, confidence_threshold)
    
    #cuda cleaning
    if device == "cuda":
        torch.cuda.empty_cache() #cuda memory was filling up
        
    
    #write the data to new directories, and switch the current dirty dataset
    rewrite_data_with_new_labels(dir_path, new_dir_path, misclassified_images)
    if current_dirty_dataset == 1:
        current_dirty_dataset = 2
    else:
        current_dirty_dataset = 1
        
    #Accuracy will be logged at new loop - loop until iterations are complete
    
    
    
    
    

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\Brandon\\Desktop\\Classes\\E208\\Homework\\E208_Final\\Automatic Data Cleaning\\Dirty Data 1'

# junk below

In [197]:
clean_dir = r"C:\Users\Brandon\Desktop\Classes\E208\Homework\E208_Final\Automatic Data Cleaning\Cropped_Data_400"
create_labels_csv(clean_dir, 'Clean_Labels.csv')

dirty_dir = r"C:\Users\Brandon\Desktop\Classes\E208\Homework\E208_Final\Automatic Data Cleaning\Dirty Data"
x = 0.5
create_dirty_data(clean_dir, dirty_dir, x)
create_labels_csv(dirty_dir, "Dirty_Labels.csv")


label_accuracy = calculate_label_accuracy("Clean_Labels.csv", "Dirty_Labels.csv")
print(f"Label accuracy of the dirty dataset: {label_accuracy}")


CSV file created at Clean_Labels.csv
Deleting Dirty Data Directory
Writing Dirty Data Directory


Writing Data: 100%|█████████████████████████████████████████████████████████████████| 385/385 [00:00<00:00, 876.39it/s]

CSV file created at Dirty_Labels.csv
Label accuracy of the dirty dataset: 0.6675324675324675


# Train a Model on the Dirty Data

In [208]:
#Dataset parameters
validation_split = 0.1
batch_size = 64
num_workers = 0
dir_path = "Dirty Data 1"

train_loader, val_loader = prepare_and_get_loaders(validation_split, batch_size, num_workers, dir_path)

In [209]:
#Model Parameters
results = {}

for arch in ["resnet18"]:
    dropout = 0
    num_classes = 3
    model = load_model(arch, dropout, num_classes)

    #Finetuning Parameters
    learning_rate = 0.0001
    num_epochs = 5
    L2 = 10**(-6)
    momentum1 = 0.8
    momentum2 = 0.9
    stepsize = 3
    gamma = 0.1
    model, hist, final_val_accuracy = finetune(model,
                                              train_loader,
                                              val_loader,
                                              learning_rate,
                                              num_epochs,
                                              L2,
                                              momentum1,
                                              momentum2,
                                              stepsize,
                                              gamma)
    results[arch] = final_val_accuracy
    print(f"Our final validation accuracy accuracy using the full CNN is {final_val_accuracy:.2f}%")

My Device is cpu


Training Epoch:  20%|█████████████▌                                                      | 1/5 [00:29<01:57, 29.40s/it]

Epoch [1/5], Training Loss: 1.1369, Validation Loss: 1.1894, Validation Accuracy: 31.58%


Training Epoch:  40%|███████████████████████████▏                                        | 2/5 [00:56<01:24, 28.01s/it]

Epoch [2/5], Training Loss: 0.6815, Validation Loss: 1.2769, Validation Accuracy: 31.58%


...

Epoch [3/5], Training Loss: 0.4524, Validation Loss: 1.3125, Validation Accuracy: 26.32%


Training Epoch:  80%|██████████████████████████████████████████████████████▍             | 4/5 [01:48<00:26, 26.71s/it]

Epoch [4/5], Training Loss: 0.3020, Validation Loss: 1.3084, Validation Accuracy: 26.32%


Training Epoch: 100%|████████████████████████████████████████████████████████████████████| 5/5 [02:14<00:00, 26.99s/it]

Epoch [5/5], Training Loss: 0.3268, Validation Loss: 1.3160, Validation Accuracy: 23.68%
Our final validation accuracy accuracy using the full CNN is 23.68%
